In [69]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(0, '../')

import pandas as pd
import xlsxwriter
import os
import os.path as osp
import numpy as np
import datetime
import datetime as dt
from dataclasses import dataclass


#my code
from load_utils import loadExcel
from sheet_number_of_travels import createSheetNumberOfTravels
from sheet_times import createSheetTimes
from sheet_number_of_cars import createSheetNumberOfCars, computeData, createFakeDataset
import yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
with open('../parameters.yaml') as f:
    params = yaml.safe_load(f)
N = params['number_of_cameras']

In [95]:
df_save = loadExcel('input.xlsx', N=params['number_of_cameras'])

In [96]:
df = df_save.copy(deep=True)
selectedDirections = [1,3,10,12]

In [97]:
writer.close()
if osp.isfile('out.xlsx'):
    os.remove('out.xlsx')

with open('../parameters.yaml') as f:
    params = yaml.safe_load(f)
    
    writer = pd.ExcelWriter('out.xlsx', engine='xlsxwriter')
    createSheetNumberOfCars(df.copy(True), writer, params)
    writer.close()


In [65]:
writer.close()

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [88]:
@dataclass
class TimeStruct:
    dateTimeStart: dt.datetime
    dateTimeEnd: dt.datetime
    sheetName: str

    @staticmethod
    def getFormat():
        return "%d.%m.%Y - %H:%M"

    @staticmethod
    def normalizeDict(dictt, df):
        ts = dictt['time_start']
        te = dictt['time_end']

        dictt['dateTime_start'] = df['Capture_time'].min().strftime('%d.%m.%Y') + " - " + ts
        dictt['dateTime_end'] = df['Capture_time'].max().strftime('%d.%m.%Y') + " - " + te

    @staticmethod
    def createFromDict(dictt, df):
        if 'dateTime_start' not in dictt:
            TimeStruct.normalizeDict(dictt, df)

        dts = dt.datetime.strptime(dictt['dateTime_start'], TimeStruct.getFormat())
        dte = dt.datetime.strptime(dictt['dateTime_end'], TimeStruct.getFormat())

        name = f'{dts.strftime("%H.%M")} -> {dte.strftime("%H.%M")}'
        return TimeStruct(dts, dte, name)

In [86]:
p = params['sheet_cars_count']['time_a']
TimeStruct.createFromDict(p, df)

TimeStruct(dateTimeStart=datetime.datetime(2021, 10, 7, 4, 50), dateTimeEnd=datetime.datetime(2021, 10, 7, 4, 50), sheetName='04.50 -> 04.50')

In [87]:
p

{'time_start': '04:50',
 'time_end': '04:50',
 'dateTime_start': '07.10.2021 - 04:50',
 'dateTime_end': '07.10.2021 - 04:50'}